In [ ]:
import os
import json
import openslide
from PIL import Image
import cv2
import numpy as np
from tqdm import tqdm
from glob import glob


In [ ]:
json_list=glob('../../data/IHC_HE_Pair_Data_GA_hospital/registration_params/*.json')
i=0
with open(json_list[i],'r') as f:
    json_data=json.load(f)

he_slide=openslide.OpenSlide(json_data['files']['he_slide'])
pdl1_slide=openslide.OpenSlide(json_data['files']['pdl1_slide'])
json_data

In [ ]:
# 썸네일 및 마스크 생성 함수
def get_thumbnail_and_mask(slide_path, downsample=30):
    """슬라이드에서 썸네일과 조직 마스크 생성"""
    slide = openslide.OpenSlide(slide_path)
    thumb = slide.get_thumbnail((slide.dimensions[0]//downsample, slide.dimensions[1]//downsample))
    thumb_np = np.array(thumb)
    
    # Grayscale 변환 및 Otsu threshold
    gray = cv2.cvtColor(thumb_np, cv2.COLOR_RGB2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # 모폴로지 연산으로 노이즈 제거
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    return slide, thumb_np, mask

# HE와 PD-L1 썸네일 및 마스크 생성
he_slide, he_thumb_np, he_mask = get_thumbnail_and_mask(json_data['files']['he_slide'])
pdl1_slide, pdl1_thumb_np, pdl1_mask = get_thumbnail_and_mask(json_data['files']['pdl1_slide'])

print(f"HE thumbnail shape: {he_thumb_np.shape}, mask tissue ratio: {np.sum(he_mask > 0) / he_mask.size:.2%}")
print(f"PD-L1 thumbnail shape: {pdl1_thumb_np.shape}, mask tissue ratio: {np.sum(pdl1_mask > 0) / pdl1_mask.size:.2%}")

In [ ]:
# HE 썸네일에 transformation 적용
from scipy.ndimage import rotate

def apply_transformation_to_thumbnail(thumb_np, mask, params):
    """썸네일과 마스크에 transformation 적용"""
    angle = params['transformation']['thumbnail']['angle_degrees']
    scale = params['transformation']['thumbnail']['scale']
    tx = int(params['transformation']['thumbnail']['translation_x'])
    ty = int(params['transformation']['thumbnail']['translation_y'])
    
    # 이미지 회전
    rotated_img = rotate(thumb_np, angle, reshape=False, order=1)
    rotated_mask = rotate(mask, angle, reshape=False, order=0)
    
    # 이미지 스케일링
    new_h, new_w = int(thumb_np.shape[0] * scale), int(thumb_np.shape[1] * scale)
    scaled_img = cv2.resize(rotated_img, (new_w, new_h))
    scaled_mask = cv2.resize(rotated_mask.astype(np.uint8), (new_w, new_h), interpolation=cv2.INTER_NEAREST)
    
    # PD-L1 크기에 맞춰 결과 이미지 생성
    target_h, target_w = params['dimensions']['pdl1_thumbnail']['height'], params['dimensions']['pdl1_thumbnail']['width']
    result_img = np.zeros((target_h, target_w, 3), dtype=np.uint8)
    result_mask = np.zeros((target_h, target_w), dtype=np.uint8)
    
    # 이동(translation) 적용
    start_y = (target_h - scaled_img.shape[0]) // 2 + ty
    start_x = (target_w - scaled_img.shape[1]) // 2 + tx
    
    src_start_y = max(0, -start_y)
    src_start_x = max(0, -start_x)
    src_end_y = min(scaled_img.shape[0], target_h - start_y)
    src_end_x = min(scaled_img.shape[1], target_w - start_x)
    
    dst_start_y = max(0, start_y)
    dst_start_x = max(0, start_x)
    dst_end_y = dst_start_y + (src_end_y - src_start_y)
    dst_end_x = dst_start_x + (src_end_x - src_start_x)
    
    if src_end_y > src_start_y and src_end_x > src_start_x:
        result_img[dst_start_y:dst_end_y, dst_start_x:dst_end_x] = scaled_img[src_start_y:src_end_y, src_start_x:src_end_x]
        result_mask[dst_start_y:dst_end_y, dst_start_x:dst_end_x] = scaled_mask[src_start_y:src_end_y, src_start_x:src_end_x]
    
    return result_img, result_mask

# HE 썸네일을 PD-L1 공간으로 변환
he_transformed_thumb, he_transformed_mask = apply_transformation_to_thumbnail(he_thumb_np, he_mask, json_data)

print(f"HE transformed shape: {he_transformed_thumb.shape}")
print(f"HE transformed mask tissue ratio: {np.sum(he_transformed_mask > 0) / he_transformed_mask.size:.2%}")

In [ ]:
# 공통 영역 추출
common_mask = np.logical_and(he_transformed_mask > 0, pdl1_mask > 0).astype(np.uint8) * 255

print(f"Common area ratio: {np.sum(common_mask > 0) / common_mask.size:.2%}")

# 시각화
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1행: 마스크
axes[0, 0].imshow(he_transformed_mask, cmap='gray')
axes[0, 0].set_title('HE Mask (Transformed)')
axes[0, 0].axis('off')

axes[0, 1].imshow(pdl1_mask, cmap='gray')
axes[0, 1].set_title('PD-L1 Mask')
axes[0, 1].axis('off')

axes[0, 2].imshow(common_mask, cmap='gray')
axes[0, 2].set_title('Common Area Mask')
axes[0, 2].axis('off')

# 2행: 오버레이
# 마스크 오버레이 (Red=HE, Blue=PD-L1, Purple=Both)
mask_overlay = np.zeros((*pdl1_mask.shape, 3), dtype=np.uint8)
mask_overlay[he_transformed_mask > 0] = [255, 0, 0]
mask_overlay[pdl1_mask > 0] = mask_overlay[pdl1_mask > 0] + [0, 0, 255]

axes[1, 0].imshow(mask_overlay)
axes[1, 0].set_title('Mask Overlay (Red=HE, Blue=PD-L1, Purple=Both)')
axes[1, 0].axis('off')

# 이미지 오버레이
alpha = 0.5
img_overlay = cv2.addWeighted(he_transformed_thumb, alpha, pdl1_thumb_np, 1-alpha, 0)
axes[1, 1].imshow(img_overlay)
axes[1, 1].set_title('Image Overlay (HE+PD-L1)')
axes[1, 1].axis('off')

# 공통영역만 표시
common_area_img = pdl1_thumb_np.copy()
common_area_img[common_mask == 0] = [255, 255, 255]  # 공통영역이 아닌 곳은 흰색
axes[1, 2].imshow(common_area_img)
axes[1, 2].set_title('Common Area Only')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 패치 좌표 변환 함수 (PD-L1 -> HE 역변환)

def get_inverse_transform_matrix(params):
    """Inverse transformation matrix 계산"""
    M = np.array(params['transformation']['fullres']['matrix'])
    M_full = np.vstack([M, [0, 0, 1]])
    M_inv = np.linalg.inv(M_full)
    return M_inv

def pdl1_to_he_transform(pdl1_x, pdl1_y, patch_size, params):
    """
    PD-L1 패치 좌표를 HE 좌표로 역변환
    회전/스케일된 영역을 포함하는 최소 외접 사각형(bounding box) 반환
    """
    M_inv = get_inverse_transform_matrix(params)
    
    # PD-L1 패치의 4개 코너 (homogeneous coordinates)
    pdl1_corners = np.array([
        [pdl1_x, pdl1_y, 1],
        [pdl1_x + patch_size, pdl1_y, 1],
        [pdl1_x + patch_size, pdl1_y + patch_size, 1],
        [pdl1_x, pdl1_y + patch_size, 1]
    ]).T  # (3, 4)
    
    # 역변환 적용
    he_corners = M_inv @ pdl1_corners  # (3, 4)
    he_corners = he_corners[:2, :].T  # (4, 2)
    
    # 변환된 영역을 포함하는 최소 외접 사각형 계산
    min_x = int(np.floor(he_corners[:, 0].min()))
    max_x = int(np.ceil(he_corners[:, 0].max()))
    min_y = int(np.floor(he_corners[:, 1].min()))
    max_y = int(np.ceil(he_corners[:, 1].max()))
    
    # 음수 좌표 처리
    min_x = max(0, min_x)
    min_y = max(0, min_y)
    
    bbox_width = max_x - min_x
    bbox_height = max_y - min_y
    
    return min_x, min_y, bbox_width, bbox_height, he_corners

# 역변환 테스트
test_pdl1_x, test_pdl1_y = 10000, 15000
patch_size = 1024

he_x, he_y, he_w, he_h, he_corners = pdl1_to_he_transform(test_pdl1_x, test_pdl1_y, patch_size, json_data)

print(f"PD-L1 patch: ({test_pdl1_x}, {test_pdl1_y}), size: {patch_size}x{patch_size}")
print(f"HE bounding box: ({he_x}, {he_y}), size: {he_w}x{he_h}")
print(f"HE corners (transformed):")
for i, corner in enumerate(he_corners):
    print(f"  Corner {i}: ({corner[0]:.1f}, {corner[1]:.1f})")


In [ ]:
# 패치 추출 함수

def get_patch_mpp(slide):
    """슬라이드의 MPP (Microns Per Pixel) 값 계산"""
    try:
        mpp_x = float(slide.properties.get('openslide.mpp-x', 0.25))
        mpp_y = float(slide.properties.get('openslide.mpp-y', 0.25))
        return (mpp_x + mpp_y) / 2
    except:
        return 0.25  # 기본값

def extract_aligned_patches(pdl1_slide, he_slide, pdl1_x, pdl1_y, patch_size, params, target_mpp=1.0):
    """
    PD-L1과 HE에서 정렬된 패치 추출
    
    Parameters:
    - pdl1_slide: PD-L1 openslide 객체
    - he_slide: HE openslide 객체
    - pdl1_x, pdl1_y: PD-L1에서의 패치 시작 좌표
    - patch_size: 패치 크기 (픽셀)
    - params: transformation 파라미터 (JSON)
    - target_mpp: 목표 MPP (기본 1.0)
    
    Returns:
    - pdl1_patch: PD-L1 패치
    - he_patch: 정렬된 HE 패치 (perspective transform 적용)
    - he_bbox: HE에서의 bounding box 정보 (x, y, w, h)
    """
    # MPP 계산
    pdl1_mpp = get_patch_mpp(pdl1_slide)
    he_mpp = get_patch_mpp(he_slide)
    
    # 실제 추출할 크기 계산 (MPP 고려)
    pdl1_extract_size = int(patch_size * target_mpp / pdl1_mpp)
    
    # 1. PD-L1 패치 추출
    pdl1_patch_raw = pdl1_slide.read_region((pdl1_x, pdl1_y), 0, (pdl1_extract_size, pdl1_extract_size))
    pdl1_patch_raw = pdl1_patch_raw.convert('RGB')
    pdl1_patch = cv2.resize(np.array(pdl1_patch_raw), (patch_size, patch_size))
    
    # 2. PD-L1 패치의 4개 코너를 HE 좌표로 역변환
    he_x, he_y, he_w, he_h, he_corners = pdl1_to_he_transform(pdl1_x, pdl1_y, pdl1_extract_size, params)
    
    # 슬라이드 범위 체크
    he_x = max(0, min(he_x, he_slide.dimensions[0] - 1))
    he_y = max(0, min(he_y, he_slide.dimensions[1] - 1))
    he_w = min(he_w, he_slide.dimensions[0] - he_x)
    he_h = min(he_h, he_slide.dimensions[1] - he_y)
    
    # 3. HE 원본에서 bounding box 전체 추출
    if he_w > 0 and he_h > 0:
        he_patch_raw = he_slide.read_region((he_x, he_y), 0, (he_w, he_h))
        he_patch_raw = np.array(he_patch_raw.convert('RGB'))
        
        # 4. he_corners를 bbox 로컬 좌표로 변환
        he_corners_local = he_corners.copy()
        he_corners_local[:, 0] -= he_x
        he_corners_local[:, 1] -= he_y
        
        # 5. Perspective Transform 적용
        # Source: HE 원본의 4개 코너 (마름모/평행사변형)
        # Destination: 정사각형
        src_pts = he_corners_local.astype(np.float32)
        dst_pts = np.array([
            [0, 0],
            [pdl1_extract_size, 0],
            [pdl1_extract_size, pdl1_extract_size],
            [0, pdl1_extract_size]
        ], dtype=np.float32)
        
        # Perspective transformation matrix 계산
        M_perspective = cv2.getPerspectiveTransform(src_pts, dst_pts)
        
        # Perspective transform 적용
        he_aligned = cv2.warpPerspective(he_patch_raw, M_perspective, 
                                         (pdl1_extract_size, pdl1_extract_size),
                                         flags=cv2.INTER_LINEAR, 
                                         borderMode=cv2.BORDER_CONSTANT, 
                                         borderValue=(255, 255, 255))
        
        # 최종 리사이즈
        he_patch = cv2.resize(he_aligned, (patch_size, patch_size))
    else:
        he_patch = np.zeros((patch_size, patch_size, 3), dtype=np.uint8)
    
    return pdl1_patch, he_patch, (he_x, he_y, he_w, he_h)

# 패치 추출 테스트
test_pdl1_x, test_pdl1_y = 87060, 45000

pdl1_patch, he_patch, he_bbox = extract_aligned_patches(
    pdl1_slide, he_slide, test_pdl1_x, test_pdl1_y, 1024, json_data, target_mpp=1.0
)

print(f"PD-L1 patch extracted: {pdl1_patch.shape}")
print(f"HE patch extracted: {he_patch.shape}")
print(f"HE bounding box: x={he_bbox[0]}, y={he_bbox[1]}, w={he_bbox[2]}, h={he_bbox[3]}")

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(pdl1_patch)
axes[0].set_title(f'PD-L1 Patch\n({test_pdl1_x}, {test_pdl1_y})')
axes[0].axis('off')

axes[1].imshow(he_patch)
axes[1].set_title(f'HE Patch (Aligned)\nBBox: ({he_bbox[0]}, {he_bbox[1]})\nSize: {he_bbox[2]}x{he_bbox[3]}')
axes[1].axis('off')

# 오버레이
alpha = 0.5
overlay = cv2.addWeighted(pdl1_patch, alpha, he_patch, 1-alpha, 0)
axes[2].imshow(overlay)
axes[2].set_title('Overlay (PD-L1 + HE)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 썸네일 상에서 패치 위치 시각화

# 테스트할 PD-L1 패치 좌표들
test_pdl1_coords = [
    (87060, 45000),
    (60000, 50000),
    (40000, 30000)
]

# 썸네일 스케일 계산
pdl1_thumb_scale_x = pdl1_thumb_np.shape[1] / pdl1_slide.dimensions[0]
pdl1_thumb_scale_y = pdl1_thumb_np.shape[0] / pdl1_slide.dimensions[1]
he_thumb_scale_x = he_thumb_np.shape[1] / he_slide.dimensions[0]
he_thumb_scale_y = he_thumb_np.shape[0] / he_slide.dimensions[1]

# 패치 크기 (MPP 고려)
patch_size_vis = 1024
target_mpp = 1.0
pdl1_mpp = get_patch_mpp(pdl1_slide)
pdl1_extract_size = int(patch_size_vis * target_mpp / pdl1_mpp)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# PD-L1 썸네일 + 패치 위치
pdl1_vis = pdl1_thumb_np.copy()
for idx, (px, py) in enumerate(test_pdl1_coords):
    # 썸네일 좌표로 변환
    thumb_x = int(px * pdl1_thumb_scale_x)
    thumb_y = int(py * pdl1_thumb_scale_y)
    thumb_size = int(pdl1_extract_size * pdl1_thumb_scale_x)
    
    # 사각형 그리기
    color = [(255, 0, 0), (0, 255, 0), (0, 0, 255)][idx % 3]
    cv2.rectangle(pdl1_vis, (thumb_x, thumb_y), 
                  (thumb_x + thumb_size, thumb_y + thumb_size), 
                  color, 3)
    # 번호 표시
    cv2.putText(pdl1_vis, f'#{idx+1}', (thumb_x + 5, thumb_y + 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

axes[0].imshow(pdl1_vis)
axes[0].set_title('PD-L1 Thumbnail\nwith Patch Locations', fontsize=14)
axes[0].axis('off')

# HE 썸네일 + 대응하는 패치 위치
he_vis = he_thumb_np.copy()
for idx, (px, py) in enumerate(test_pdl1_coords):
    # PD-L1 → HE 변환
    he_x, he_y, he_w, he_h, he_corners = pdl1_to_he_transform(px, py, pdl1_extract_size, json_data)
    
    # 썸네일 좌표로 변환
    he_corners_thumb = he_corners.copy()
    he_corners_thumb[:, 0] = he_corners_thumb[:, 0] * he_thumb_scale_x
    he_corners_thumb[:, 1] = he_corners_thumb[:, 1] * he_thumb_scale_y
    
    # 변환된 사각형(실제로는 회전된 사각형) 그리기
    color = [(255, 0, 0), (0, 255, 0), (0, 0, 255)][idx % 3]
    pts = he_corners_thumb.astype(np.int32).reshape((-1, 1, 2))
    cv2.polylines(he_vis, [pts], True, color, 3)
    
    # 번호 표시
    center_x = int(he_corners_thumb[:, 0].mean())
    center_y = int(he_corners_thumb[:, 1].mean())
    cv2.putText(he_vis, f'#{idx+1}', (center_x - 20, center_y), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

axes[1].imshow(he_vis)
axes[1].set_title('HE Thumbnail\nwith Aligned Patch Locations', fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"패치 위치 비교:")
for idx, (px, py) in enumerate(test_pdl1_coords):
    he_x, he_y, he_w, he_h, he_corners = pdl1_to_he_transform(px, py, pdl1_extract_size, json_data)
    print(f"  Patch #{idx+1}: PD-L1({px}, {py}) → HE bbox({he_x}, {he_y}, {he_w}x{he_h})")


In [ ]:
# 각 패치 위치에서 실제 패치 추출 및 정렬 확인

fig, axes = plt.subplots(len(test_pdl1_coords), 3, figsize=(18, 6 * len(test_pdl1_coords)))

for idx, (px, py) in enumerate(test_pdl1_coords):
    # 패치 추출
    pdl1_patch, he_patch, he_bbox = extract_aligned_patches(
        pdl1_slide, he_slide, px, py, 1024, json_data, target_mpp=1.0
    )
    
    # 시각화
    axes[idx, 0].imshow(pdl1_patch)
    axes[idx, 0].set_title(f'Patch #{idx+1}: PD-L1\n({px}, {py})', fontsize=12)
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(he_patch)
    axes[idx, 1].set_title(f'HE (Aligned)\nBBox: ({he_bbox[0]}, {he_bbox[1]})', fontsize=12)
    axes[idx, 1].axis('off')
    
    # 오버레이
    alpha = 0.5
    overlay = cv2.addWeighted(pdl1_patch, alpha, he_patch, 1-alpha, 0)
    axes[idx, 2].imshow(overlay)
    axes[idx, 2].set_title(f'Overlay (PD-L1 + HE)', fontsize=12)
    axes[idx, 2].axis('off')

plt.tight_layout()
plt.show()
